In [3]:
#import numpy as np
#from snorer import Constants#radiusInfluence,radiusSchwarzschild,co
#import matplotlib.pyplot as plt
from fractions import Fraction

In [ ]:
class HaloSpike(Constants):

    def __init__(self,mBH,tBH,alpha):
        self._mBH = mBH
        self.tBH = tBH
        self._alpha = self._check_alpha(alpha)

        # Quantities to be evaluated
        self._rh = None
        self._Rs = None
        self._norm = None

    def __repr__(self):
        return '{:>18s}'.format('SMBH mass:') + ' {:>.3e} M_sun'.format(self.mBH) + '\n' +                 \
               '{:>18s}'.format('Spike slope:') + ' {}'.format(self._alpha) + '\n' + \
               '{:>18s}'.format('Spike radius:') + ' {:>.3e} kpc'.format(self.Rsp) + '\n' +              \
               '{:>18s}'.format('Influence radius:') + ' {:>.3e} kpc'.format(self.rh)

    # mBH property and setter
    @property
    def mBH(self):
        return self._mBH

    @mBH.setter
    def mBH(self,new_mBH):
        self._mBH = new_mBH
        self._Rs = None # Set it to None will make Rs re-evaluated with new mBH
        self._norm = None # Set it to None will make norm re-evaluated with new mBH
    
    # Schwarzschild radius
    @property
    def Rs(self):
        if self._Rs is None:
            # mBH is updated, evlaute the corresponding new Rs
            self._Rs = radiusSchwarzschild(self.mBH)
        return self._Rs

    # Influence radius property
    @property
    def rh(self):
        if self._rh is None:
            # mBH is updated, evlaute the corresponding new rh
            self._rh = radiusInfluence(self.mBH)
        return self._rh

    @rh.setter(self,new_rh):
    def rh(self,new_rh):
        self._rh = new_rh
        self._norm = None # Set it to None will make norm re-evaluated with new rh
    
    # Spike index
    @property
    def alpha(self):
        return self._alpha

    @alpha.setter
    def alpha(self,new_alpha):
        self._alpha = self._check_alpha(new_alpha)
        self._norm = None # Set it to None will make norm re-evaluated with new alpha

    # Normalization factor
    @propetry
    def norm(self):
        if self._norm is None:
            self._norm = self._normN()
        return self._norm

    def _check_alpha(self,alpha) -> str:
        if alpha in ('3/2','7/3'):
            return alpha
        else:
            raise FlagError('Spike index \'alpha\' must be either [\'3/2\' or \'7/3\'].')

    def _normN(self) -> float:
        """
        The normalization N
        
        Output
        ------
        normalization N
        """
        # ini parameters
        mBH = self.mBH
        rh = self.rh
        Rs = self.Rs
        ri = 4*Rs
        alpha = eval(self.alpha)
        
        def _fa(r,alpha):
            return (r**3 / (3 - alpha) + 12 * Rs * r**2 / (alpha - 2) - 48 * Rs**2 * r / (alpha - 1) + 64 * Rs**3 / alpha) / r**alpha

        fh,fi = _fa(rh,alpha),_fa(ri,alpha)
        norm = mBH * self.Msun / 4 / pi / (fh - fi)
        return norm

    def _radiusSpike(self,rhos,rs) -> float:
        """
        Get the spike radius

        Input
        ------
        rhos: characteristic density, MeV/cm^3
        rs: characteristic radius, kpc

        Output
        ------
        R_spike: Spike radius, kpc
        """
        # Parameter
        rh = self.rh
        alpha = eval(self.alpha)
        norm = self.norm
        rhos = rhos * self.kpc2cm**3
        if self.alpha == '3/2':
            return (norm / rhos / rs)**(3/4) * rh**(5/8)
        elif self.alpha == '7/3':
            return (norm / rhos / rs)**alpha

    def _rhoPrime(self,r,Rsp,norm) -> float:
        """
        Get rho'

        Input
        ------
        r: distance to GC, kpc
        Rsp: Spike radius, kpc
        norm: Normalization factor

        Output
        ------
        rhoPrime: MeV/cm^3
        """
        # Parameters
        Rs = self.Rs
        rh = self.rh
        alpha = eval(self.alpha)
        ri = 4 * Rs

        if self.alpha == '3/2':
            rhoN = norm / rh**alpha
            rhoNp = rhoN * (rh / Rsp)**(7/3)
            if ri <= r < rh:
                rhoP = rhoN * (1 - ri / r)**3 * (rh / r)**alpha
            else:
                rhoP = rhoNp * (Rsp / r)**(7 / 3)
            return rhoP / self.kpc2cm**3
        elif self.alpha == '7/3':
            rhoN = norm / Rsp**(7/3)
            rhoP = rhoN * (1 - 4 * Rs / r)**3 * (Rsp / r)**alpha
            return rhoP / self.kpc2cm**3

    def _nxSpike(self,r,mx,sigv,rhos,rs,n) -> float:
        """
        DM number density with spike in the center

        Input
        ------
        r: distance to GC, kpc
        mx: DM mass, MeV
        sigv: DM annihilation cross section, in the unit of 1e-26 cm^3/s
            None indicates no annihilation
        rhos: The characteristic density, MeV/cm^3
        rs: The characteristic radius, kpc
        n: Slope of the DM profile

        Output
        ------
        number density: #/cm^3
        """
        # Parameters
        Rs = self.Rs
        Rsp = self._radiusSpike(rhos,rs)
        norm = self.norm
        tBH = self.tBH
        ri = 4*Rs
        
        if sigv is None:  # no DM annihilation
            if r < ri:
                return 0
            elif ri <= r < Rsp:
                return self._rhoPrime(r,Rsp,norm) / mx
            else:
                return rhox(r,rhos,rs,n)/mx
        else: # non-zero DM annihilation
            sigv = sigv * 1e-26
            rhoc = mx / sigv / tBH / self.year2Seconds
            if r < ri:
                return 0
            elif ri <= r < Rsp:
                rhoP = self._rhoPrime(r,Rsp,norm)
                return rhoP * rhoc / (rhoP + rhoc) / mx
            else:
                rhoDM = rhox(r,rhos,rs,n)
                return rhoDM * rhoc / (rhoDM + rhoc) / mx
                
    def __call__(self,r,mx,sigv,rhos,rs,n) -> float:
        """
        Obtain the DM number density in the presence of spike
        
        Input
        ------
        r: distance to GC, kpc
        mx: DM mass, MeV
        sigv: DM annihilation cross section, in the unit of 1e-26 cm^3/s
            None indicates no annihilation
        rhos: The characteristic density, MeV/cm^3
        rs: The characteristic radius, kpc
        n: Slope of the DM profile

        Output
        ------
        number density: #/cm^3
        """
        return self._nxSpike(r,mx,sigv,rhos,rs,n)

In [5]:
eval('3/2')

1.5

In [7]:
N?

Object `N` not found.
